In [1]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [2]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision.transforms import Resize, ToTensor, Normalize

from tqdm import tqdm

In [3]:
import sys
import gzip
import random
import platform
import warnings
import collections

import re
import requests
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import torchvision
import torch.optim as optim
from torchvision import transforms, utils

from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler
from torchvision.transforms.functional import to_pil_image


# Train 과 Val split 해서 각자의 labeled된 csv 생성

In [4]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import glob
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import glob
from tqdm import tqdm

In [5]:
def makeLabeling(pd,idx,name):
    if name == 'mask':
        if pd['gender'][idx] == 'male':
            if pd['age'][idx] < 30: return 0
            elif 30 <= pd['age'][idx] < 60: return 1
            else: return 2
        else:
            if pd['age'][idx] < 30: return 3
            elif 30 <= pd['age'][idx] < 60: return 4
            else: return 5
    elif name == 'incorrect':
        if pd['gender'][idx] == 'male':
            if pd['age'][idx] < 30: return 6
            elif 30 <= pd['age'][idx] < 60: return 7
            else: return 8
        else:
            if pd['age'][idx] < 30: return 9
            elif 30 <= pd['age'][idx] < 60: return 10
            else: return 11
    else:
        if pd['gender'][idx] == 'male':
            if pd['age'][idx] < 30: return 12
            elif 30 <= pd['age'][idx] < 60: return 13
            else: return 14
        else:
            if pd['age'][idx] < 30: return 15
            elif 30 <= pd['age'][idx] < 60: return 16
            else: return 17

In [24]:
train_csv = pd.read_csv('/opt/ml/input/data/train/train.csv')


# val train split 할 기준 세우기
import random
total = list(range(2700))
validx = random.sample(total,int(2700*0.2))
validx.sort()
print(len(validx))
trainidx = total
for i in validx:
    trainidx.remove(i)
print(len(trainidx))

540
2160


In [27]:
v_file_dict = {k:[] for k in ['id','path','file_name','absolute_path','label']}

IMAGE_PATH = '/opt/ml/input/data/train/images'

# val_img_label
for i in validx:
    total_path = os.path.join(IMAGE_PATH,str(train_csv['path'][i]))
    for file_name in os.listdir(total_path):
        # 확장자 제거, 이름만 가져오기
        fn_split = file_name.split(".")
        if len(fn_split[0]) == 0: 
            continue
        name = re.findall("[a-z]+",fn_split[0])[0]
        
        # labeling
        label = makeLabeling(train_csv,i,name)
        v_file_dict['id'].append(train_csv['id'][i])
        v_file_dict['file_name'].append(file_name)
        v_file_dict['path'].append(train_csv['path'][i])
        v_file_dict['absolute_path'].append(os.path.join(total_path,file_name))
        v_file_dict['label'].append(label)

for k in v_file_dict:
    print(f"{k}: {len(v_file_dict[k])}")


id: 9290
path: 9290
file_name: 9290
absolute_path: 9290
label: 9290


In [28]:
image_label = pd.DataFrame(v_file_dict)
np.unique(image_label['label'])
image_label.to_csv('val_image_label.csv',index=False)

In [8]:
#아래부터는 train csv 만드는 과정

In [29]:
t_file_dict = {k:[] for k in ['id','path','file_name','absolute_path','label']}

IMAGE_PATH = '/opt/ml/input/data/train/images'

# val_img_label
for i in trainidx:
    total_path = os.path.join(IMAGE_PATH,str(train_csv['path'][i]))
    for file_name in os.listdir(total_path):
        # 확장자 제거, 이름만 가져오기
        fn_split = file_name.split(".")
        if len(fn_split[0]) == 0: 
            continue
        name = re.findall("[a-z]+",fn_split[0])[0]
        
        # labeling
        label = makeLabeling(train_csv,i,name)
        t_file_dict['id'].append(train_csv['id'][i])
        t_file_dict['file_name'].append(file_name)
        t_file_dict['path'].append(train_csv['path'][i])
        t_file_dict['absolute_path'].append(os.path.join(total_path,file_name))
        t_file_dict['label'].append(label)

for k in t_file_dict:
    print(f"{k}: {len(t_file_dict[k])}")


id: 37236
path: 37236
file_name: 37236
absolute_path: 37236
label: 37236


In [30]:
image_label = pd.DataFrame(t_file_dict)
np.unique(image_label['label'])
image_label.to_csv('train_image_label.csv',index=False)

# 생성된 csv 라벨 개수 비교

In [31]:
read_val = pd.read_csv('/opt/ml/my_augmentation/val_image_label.csv')
read_train = pd.read_csv('/opt/ml/my_augmentation/train_image_label.csv')

val_label = list(read_val['label'])
train_label = list(read_train['label'])

v_lab_count = [0 for i in range(18)]
t_lab_count = [0 for i in range(18)]
# 퍼센트로 변경
v_lab_per = [0 for i in range(18)]
t_lab_per = [0 for i in range(18)]

for i in val_label:
    v_lab_count[int(i)] += 1

for i, j in enumerate(v_lab_count):
    v_lab_per[i] = j/len(val_label)
    
for i in train_label:
    t_lab_count[int(i)] += 1

for i, j in enumerate(t_lab_count):
    t_lab_per[i] = j/len(train_label)

print("val 개수, train 개수 = ",len(val_label), len(train_label))
print("")
print("label 비율")
print("----------------------")

print(v_lab_per)
print("----------------------")
print(t_lab_per)





val 개수, train 개수 =  9290 37236

label 비율
----------------------
[0.11141011840688912, 0.08503767491926803, 0.045748116254036596, 0.15231431646932186, 0.17330462863293863, 0.06189451022604951, 0.03240043057050592, 0.025511302475780408, 0.012809472551130248, 0.04510226049515608, 0.051991388589881594, 0.017330462863293865, 0.03240043057050592, 0.025511302475780408, 0.012809472551130248, 0.04510226049515608, 0.051991388589881594, 0.017330462863293865]
----------------------
[0.11102159200773445, 0.08942958427328392, 0.04632613599742185, 0.15197658180255666, 0.17402513696422817, 0.056397035127296165, 0.03246857879471479, 0.026828875281985177, 0.012971318079278118, 0.04514448383284993, 0.05220754108926845, 0.015791169835642927, 0.03246857879471479, 0.026828875281985177, 0.012971318079278118, 0.04514448383284993, 0.05220754108926845, 0.015791169835642927]
